In [ ]:
import functools
import warnings

from sklearn import metrics
import numpy as np
from classes.boosting_matrix import BoostingMatrix
from classes.dataset import Dataset
from settings import Settings
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import Counter
from classes.pattern_boosting import PatternBoosting
from settings import Settings
from classes.enumeration.estimation_type import EstimationType
from data.synthetic_dataset import SyntheticDataset
import pandas as pd
import copy
from classes.analysis_patternboosting import AnalysisPatternBoosting
from data.load_dataset import load_dataset
from data import data_reader
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from multiprocessing.dummy import Pool as ThreadPool

import copy
from data import data_reader
from classes.wrapper_pattern_boosting import WrapperPatternBoosting
from jupiter_notebook_functions import *
from classes.wrapper_pattern_boosting import WrapperPatternBoosting
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from typing import List, Tuple, Optional
import warnings

Settings:

In [ ]:
dataset_name = "5_k_selection_graphs"

# the model will be searched in the directory results/jupiter
pattern_boosting_model_name = "frequency_matrix"

n_learners = 300
number_of_learners = [1, 5, 10, 15, 20, 25, 35] + list(range(50, n_learners + 1, 50))

max_path_length = [1, 2, 3, 4, 5, 15]

max_depth = 5

errors_test_list = []
errors_train_list = []
n_learners_list = []
labels_list = []

print("Done")

In [ ]:
# Load the dataset with the matrix with all possible paths

directory = data_reader.get_save_location(folder_relative_path="results/jupiter")
pattern_boosting = data_reader.load_data(directory=directory, filename="pattern_boosting_all_paths")

dataset = load_dataset(dataset_name)

# get the frequency matrix
print("get the frequency matrix")
'''
if isinstance(pattern_boosting, PatternBoosting):
    frequency_matrix = pattern_boosting.create_boosting_matrix_for(dataset)

elif isinstance(pattern_boosting, WrapperPatternBoosting):
    frequency_matrix = pattern_boosting.create_ordered_boosting_matrix(dataset)
'''
frequency_matrix = pd.DataFrame(pattern_boosting.boosting_matrix.get_matrix(),
                                columns=pattern_boosting.boosting_matrix.get_header())

print(len(pattern_boosting.boosting_matrix.get_header()))

# run classical XGB with depth 1 using different number of learners, parallelize over the max length paths
print("run classical XGB with depth 1 using different number of learners")
pool = ThreadPool(min(10, len(max_path_length)))

xgb_err = pool.map(
    functools.partial(get_XGB_error_and_variable_importance,
                      frequency_matrix=copy.deepcopy(frequency_matrix),
                      labels=dataset.labels,
                      max_number_of_learners=number_of_learners),
    max_path_length)

xgb_test_err, xgb_train_err, variable_importance = zip(*xgb_err)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Run xgb full power and on the frequency matrix with all possible paths
print("Run xgb full power and on the frequency matrix with all possible paths")
xgb_settings = Settings.xgb_model_parameters
xgb_settings['max_depth'] = max_depth
test_err_full_power_xgb, train_err_full_power_xgb, _ = get_XGB_error_and_variable_importance(
    max_path_length=200,
    frequency_matrix=copy.deepcopy(frequency_matrix),
    labels=dataset.labels,
    max_number_of_learners=number_of_learners,
    xgb_settings=xgb_settings)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Run xgb full power and on the frequency matrix given by pattern boosting
print("Run xgb full power and on the frequency matrix given by pattern boosting")
# load pattern boosting model, to use the headers found by the algorithm
directory = data_reader.get_save_location(folder_relative_path="results/jupiter")
pattern_boosting = data_reader.load_data(directory=directory, filename="pattern_boosting_500_steps")
print(pattern_boosting.test_dataset)

pd_pattern_boosting_matrix = pd.DataFrame(pattern_boosting.boosting_matrix.get_matrix(),
                                          columns=pattern_boosting.boosting_matrix.get_header())

test_err_full_power_xgb_on_pattern_boosting_matrix, train_err_full_power_xgb_on_pattern_boosting_matrix, _ = get_XGB_error_and_variable_importance_t(
    max_path_length=200,
    pattern_boosting=pattern_boosting,
    max_number_of_learners=number_of_learners,
    frequency_matrix=pd_pattern_boosting_matrix,
    xgb_settings=xgb_settings)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#merging all the errors

errors_test_list += xgb_test_err
errors_train_list += xgb_train_err
for length in max_path_length:
    n_learners_list.append(number_of_learners)
    labels_list.append('XGB path ' + str(length))

errors_test_list.append(test_err_full_power_xgb)
errors_train_list.append(test_err_full_power_xgb)
n_learners_list.append(number_of_learners)
labels_list.append('XGB depth ' + str(max_depth) + ' on all paths')

errors_test_list.append(test_err_full_power_xgb)
errors_train_list.append(test_err_full_power_xgb)
n_learners_list.append(number_of_learners)
labels_list.append('full XGB pattern boosting matrix')
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


errors_test_list.append(pattern_boosting.test_error)
errors_train_list.append(pattern_boosting.train_error)
n_learners_list.append(range(1, len(pattern_boosting.test_error) + 1))
labels_list.append('pattern boosting')
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# saving the errors
try:
    old_n_learners = data_reader.load_data(directory="results/jupiter/plot", filename="old_n_learners")
except:
    warnings.warn("Old number of learners not found")
    old_n_learners = 0
if n_learners > old_n_learners:
    data_reader.save_data(n_learners, filename="old_n_learners", directory="results/jupiter/plot",
                          create_unique_subfolder=False)
    data_reader.save_data(errors_test_list, filename="errors_test_list", directory="results/jupiter/plot")
    data_reader.save_data(errors_train_list, filename="errors_train_list", directory="results/jupiter/plot")
    data_reader.save_data(n_learners_list, filename="n_learners_list", directory="results/jupiter/plot")
    data_reader.save_data(labels_list, filename="labels_list", directory="results/jupiter/plot")

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




max path length:  15
max path length:  15
max path length:  15
Run xgb full power and on the frequency matrix with all possible paths
Learner number:  1
Learner number:  5
Learner number:  10
Learner number:  15
Learner number:  20
Learner number:  25
Learner number:  35
Learner number:  50
Learner number:  100
Learner number:  150
Learner number:  200
Learner number:  250


In [ ]:
# load the errors
directory = data_reader.get_save_location(folder_relative_path="results/jupiter", create_unique_subfolder=True)
errors_test_list = data_reader.load_data(directory=directory, filename="errors_test_list")
errors_train_list = data_reader.load_data(directory=directory, filename="errors_train_list")
n_learners_list = data_reader.load_data(directory=directory, filename="n_learners_list")
labels_list = data_reader.load_data(directory=directory, filename="labels_list")

directory = '/Users/popcorn/PycharmProjects/pattern_boosting/results/Xgb_step_200_max_path_length_100_5_k_selection_graphs/wrapped_boosting/'

wrapper_pattern_boosting = data_reader.load_data(directory=directory, filename="wrapper_pattern_boosting")

errors_test_list.append(wrapper_pattern_boosting.test_error)
errors_train_list.append(wrapper_pattern_boosting.train_error)
n_learners_list.append(range(1, len(wrapper_pattern_boosting.test_error) + 1))
labels_list.append('wrapper pattern boosting')


#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


def plot_error_evolution(error_list: List[List[float]],
                         number_of_learners_list: List[List[float]],
                         labels: List[str],
                         title: str = 'Evolution of the Error',
                         y_lim: Optional[Tuple[float, float]] = None) -> None:
    """
    Plots the evolution of different error series based on corresponding number of learners and error values.

    :param error_list: A list of lists, where each sublist contains the error values for a different model or condition.
    :type error_list: list[list[float]]
    :param number_of_learners_list: A list of lists of numbers representing learners that correspond to the error values.
    :type number_of_learners_list: list[list[float]]
    :param labels: A list of strings that serve as labels for the plots. Must be the same length as error_list.
    :type labels: list[str]
    :param title: The title of the plot. Defaults to 'Evolution of the Error' if not provided.
    :type title: str
    :param y_lim: An optional tuple with two float values to set the limits of the y-axis. Defaults to None if not provided.
    :type y_lim: tuple[float, float], optional
    :return: None
    """

    if not error_list or not number_of_learners_list or not labels:
        raise ValueError("error_list, number_of_learners_list, and labels must not be empty.")

    if len(error_list) != len(labels) or len(error_list) != len(number_of_learners_list):
        raise ValueError(
            f"The number of labels ({len(labels)}) and the number of error/value series ({len(error_list)}) must match.")

    for error_values, number_of_learners_values in zip(error_list, number_of_learners_list):
        if len(error_values) != len(number_of_learners_values):
            raise ValueError("Each error sublist must be of the same length as its corresponding value sublist.")

    plt.style.use('ggplot')  # Set the plot style to 'ggplot'

    fig, ax = plt.subplots()

    for error, number_of_learners, label in zip(error_list, number_of_learners_list, labels):
        ax.plot(number_of_learners, error, label=label)

    ax.xaxis.set_major_locator(MaxNLocator(integer=True))  # Force x-axis to show only integer values

    if y_lim is not None:
        ax.set_ylim(y_lim)

    ax.set_xlabel('Number of Learners')
    ax.set_ylabel('Error')
    ax.set_title(title)
    ax.legend()
    ax.grid(True)

    plt.show()


plot_error_evolution(errors_test_list, n_learners_list, labels_list, title="Test Error", y_lim=(0.00075, 0.00125))
plot_error_evolution(errors_train_list, n_learners_list, labels_list, title="Train Error", y_lim=(0.00075, 0.00125))
print("done")
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

